In [43]:
import pickle
import numpy as np

In [44]:
with open("train_qa.txt","rb") as f:
    train_data = pickle.load(f)

In [45]:
with open("test_qa.txt","rb") as f:
    test_data = pickle.load(f)

In [46]:
type(train_data)

list

In [47]:
type(test_data)

list

In [48]:
len(train_data)

10000

In [49]:
len(test_data)

1000

In [50]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [51]:
" ".join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [52]:
" ".join(train_data[0][1])

'Is Sandra in the hallway ?'

In [53]:
train_data[0][2]

'no'

In [54]:
vocab = set()

In [55]:
all_data = train_data + test_data

In [56]:
for story, question, answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [57]:
vocab.add("no")
vocab.add("yes")

In [58]:
vocab_len = len(vocab)+1

In [59]:
max_story_len = max([len(data[0]) for data in all_data])

In [60]:
max_ques_len = max([len(data[1]) for data in all_data])

In [61]:
max_ques_len

6

In [62]:
vocab_size = len(vocab) + 1

In [63]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [64]:
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [65]:
tokenizer.word_index

{'put': 1,
 'took': 2,
 'the': 3,
 'hallway': 4,
 'john': 5,
 'grabbed': 6,
 'down': 7,
 'journeyed': 8,
 'travelled': 9,
 'office': 10,
 'mary': 11,
 'bedroom': 12,
 'apple': 13,
 'discarded': 14,
 'back': 15,
 'garden': 16,
 '?': 17,
 'kitchen': 18,
 'daniel': 19,
 'bathroom': 20,
 'to': 21,
 'football': 22,
 'went': 23,
 'got': 24,
 'there': 25,
 'yes': 26,
 'dropped': 27,
 'moved': 28,
 'milk': 29,
 '.': 30,
 'picked': 31,
 'up': 32,
 'sandra': 33,
 'no': 34,
 'is': 35,
 'left': 36,
 'in': 37}

In [66]:
train_story_text =[]
train_question_text =[]
train_answers =[]

for story, question, ans in train_data:
    train_story_text.append(story)
    train_question_text.append(question)

In [67]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [68]:
len(train_story_text)

10000

In [69]:
len(train_story_seq)

10000

In [70]:
len(tokenizer.word_index)

37

In [71]:
word_index = tokenizer.word_index

In [76]:
def vectorize_stories(data, word_index = tokenizer.word_index, max_story_len = max_story_len, max_ques_len = max_ques_len):
    '''
    INPUT:

    data: consisting of stories, questions and answers
    word_index: word index dictionary from tokenizer
    max_story_len: the length of the longest story
    max_question_len: the length of the longest ques

    OUPUT:

    Vectorizes the stories, questions and queries into the padded sequences. we first loop for every story, query, answer in the data. 
    Then we convert the raw words to an word index value. Then we append each set to their appropriate output list. 
    Then once we have converted the words to numbers, we pad the sequences so they all are of equal length.

    Returns this in the form of a tuple (X,Xq,Y)(padded based on max lengths)
    '''
    X=[]
    Xq=[]
    Y=[]
    for story, query, answer in data:
        x= [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in query]
        y = np.zeros(len(word_index)+1)
        y[word_index[answer]]=1

        X.append(x)
        Xq.append(xq)
        Y.append(y)

    return (pad_sequences(X, maxlen = max_story_len), pad_sequences(Xq,maxlen = max_ques_len), np.array(Y))

In [77]:
inputs_train, queries_train, answer_train = vectorize_stories(train_data)

In [80]:
inputs_test, queries_test, answer_test = vectorize_stories(test_data)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Input, Activation, Dense, Permute, Dropout